In [1]:
!pip install BeautifulSoup4
!pip install requests



    100% |████████████████████████████████| 102kB 1.3MB/s a 0:00:01


In [4]:

class Advert:
    def __init__(self, id: str, title: str, price: int, reg_year_month: str, body_type: str, miles: int, cc: float, transmission: str, fuel: str, url: str):
        self.id = id
        self.title = title
        self.price = price
        self.reg_year_month = reg_year_month
        self.body_type = body_type
        self.miles = miles
        self.cc = cc
        self.transmission = transmission
        self.fuel = fuel
        self.url = url 
        self.descriptionRaw = ""
        self.other1 = ""
        self.other2 = ""
        self.other3 = ""
        
        
import requests
from bs4 import BeautifulSoup

spec_list = []

In [7]:
print("345")

## initialise
car_count = 0

page_count = 0
page = 1
site_domain = "https://www.autotrader.co.uk"

search_url = "/car-search"
detail_url ="/classified/advert/"
query_param = "?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&year-to=2019&body-type=Coupe&transmission=Automatic"
#query_param = "?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=Q5&transmission=Automatic"
has_more_pages = True
while has_more_pages :
    # request a page
    url  =  site_domain + search_url + query_param + "&page=" + str(page)
    print(url)
    response = requests.get( url )
    soup = BeautifulSoup(response.text, 'html.parser')

    # did we fi d any ad's
    
    car_count = int(soup.find('h1', class_="search-form__count").text.split(" ")[0])
    if len(spec_list) == car_count:
        print("PageCount: {}, SpecList: {}".format(page, len(spec_list)) )
        # No ad's found - stop
        break
        
        
    page_count = int( (car_count + 5) / 10 )

    print("Processing Ad's from page {} of {}. For total of Car count: {}".format(page, page_count, car_count) )
    adverts = soup.find_all('article', class_="search-listing") 

    # Process Ad's on current page
    ad_number = 0
    for advert in adverts :
        ad_number += 1
        price = advert.find('div', class_="vehicle-price").text.replace("£","").replace(",","")
        
        info_div = advert.find( 'div', class_="information-container")
        title = info_div.find( 'h2', class_="listing-title").text
        ad_url = info_div.find( 'a', class_="listing-fpa-link")['href']
        ad_id = ad_url.split("/")[3].split("?")[0]
        
        ## Check fro dups
        is_dup = False
        for exist_ad in spec_list:
            #print("id- {} dup check {} ".format(ad_id, str(exist_ad) ) )
            if exist_ad.id == str( ad_id):
                is_dup = True
                break
        if is_dup:
            print("skipping duplicate add for ad_id "+ ad_id)
            continue
            
        
        spec_tags = info_div.find( 'ul',  class_="listing-key-specs").findChildren()

        ##print("Page: {}, ad: {}, parse spec structure: {}".format(page, ad_number, spec_tags))
        raw_year = spec_tags[0].text.split(" ")
        raw_bodyType = spec_tags[1].text.lower()
        raw_miles = spec_tags[2].text.replace(",","").split(" ") 
        raw_cc = spec_tags[3].text.replace("L","")
        ## this is horrid
        ## 1 2 3 4 5 6
        ## 1 2 3 5 6       <- for some ad's the bhp was missing
        if ( spec_tags[4].text[-3:] == "bhp"):
            raw_bhp = spec_tags[4].text.split(" ")
            raw_transmission = spec_tags[5].text.lower()
            raw_fuel = spec_tags[6].text.lower()
        else:
            raw_bhp = ""
            raw_transmission = spec_tags[4].text.lower()
            raw_fuel = spec_tags[5].text.lower()
            
        spec_list.append( Advert(ad_id, title, int(price), raw_year[0]+raw_year[1].replace("(","/"), raw_bodyType, int(raw_miles[0]), float(raw_cc),  raw_transmission, raw_fuel, site_domain+detail_url+ad_id) )

    
       
        
    if page == page_count :
        has_more_pages = False
    else:
        page += 1
    
    # end of page processing, next page?                   

# end of Site scrape
print("Found {} ad specs".format( len(spec_list)) )

print("<253.")

345
https://www.autotrader.co.uk/car-search?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&year-to=2019&body-type=Coupe&transmission=Automatic&page=1
Processing Ad's from page 1 of 9. For total of Car count: 93
https://www.autotrader.co.uk/car-search?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&year-to=2019&body-type=Coupe&transmission=Automatic&page=2
Processing Ad's from page 2 of 9. For total of Car count: 93
skipping duplicate add for ad_id 201909122142555
https://www.autotrader.co.uk/car-search?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&year-to=2019&body-type=Coupe&transmission=Automatic&page=3
Processing Ad's from page 3 of 9. For total of Car count: 93
skipping duplicate add for ad_id 201909122142555
https://www.autotrader.co.uk/car-search?sort=price-asc&radius=200&postcode=tn278eh&onesearchad=Used&make=AUDI&model=R8&year-from=2007&ye

In [8]:

## get individual ads for car images and details
ad_number = 0
for advert in spec_list:
    ad_number += 1
    print("inspection of Advert {} of {} for more details".format(ad_number, len(spec_list)) )
    print("url: " + advert.url)
    car_detail_response = requests.get( advert.url )
    soup_detail = BeautifulSoup(car_detail_response.text, 'html.parser')
    ##print(soup_detail)
    
    #<button type="button" class="gallery-thumbs__placeholder"><img src="https://m.atcdn.co.uk/a/media/w100h75/8a84642bf4a24ee1887b101ad68b4ede.jpg" alt=" "></button>
    
    meta_description = soup_detail.find('meta',attrs={'name':'og:description'})
    advert.descriptionRaw = meta_description["content"]
    
    print("Meta detial: #",ad_number," id:",advert.id,"  desc:",advert.descriptionRaw)
        
    
    
        
    
    

inspection of Advert 1 of 90 for more details
url: https://www.autotrader.co.uk/classified/advert/201911104213220
Meta detial: # 1  id: 201911104213220   desc: Price: £29,995. Just Service at 58K ( £2,435) Full Service History All Previous MOTs (MOT May 2020)Full Carbon Pack Interior Upholstery upgrade (Kream Developments) Wrapped in satin black  Factory ibis white underneath great condition  (See last photos ) All service paperwork and bills .R8DHO reg included, Electric windows, Air conditioning, Satellite navigation, Parking aid, CD player, Bluetooth, Leather trim, Heated seats, Height adjustable driver's seat, Height adjustable passenger seat, 21" Alloy Wheels, Power steering, Steering wheel rake adjustment, Steering wheel reach adjustment, Cruise control, Traction control, Central locking, Alarm, Driver's airbags, Immobiliser, Side airbags, Passenger airbags. White, 5+ owners, £29,995
inspection of Advert 2 of 90 for more details
url: https://www.autotrader.co.uk/classified/advert

Meta detial: # 11  id: 201909122142555   desc: Price: £64,995. GKirby are proud to offer this stunning R8 V10 Plus S-Tronic. Ibis White, Fine Black Nappa Leather Comfort Seats With Diamond Cross Stitch, Carbon Fibre Interior Trim, 19" 5 Double Spoke Y Alloys In Titanium, Carbon Fibre Front and Rear Diffusers, Carbon Fibre Wing Mirrors, Audi Parking Plus, Parking Sensors Front and Rear, LED Headlights, Illuminated Door Sills, Engine Bay Lighting, Carbon Fibre Engine Bay, Heated Seats, Electric Seats, Cruise Control, Air Conditioning, Audi Navigation System, Bluetooth Telephone Prep, Bang & Olufsen Sound System. Full Audi Service History, 4 Nearly New Michelin Pilot Sport Cup 2s
inspection of Advert 12 of 90 for more details
url: https://www.autotrader.co.uk/classified/advert/201910093128237
Meta detial: # 12  id: 201910093128237   desc: Price: £35,000. This is an amazing car with so many extras including a specialist super loud exhaust system. Sounds like heaven! Carbon fibre in loads o

Meta detial: # 17  id: 201909302778502   desc: Price: £36,975. 0% FINANCE AVAILABLE! HPI CLEAR!! Junction 17 Cars now have to offer a stunning example Audi R8 4.2 V8 R-Tronic Quattro 2dr Coupe.. Outstanding Audi dealer service history, with exceptional low mileage.. Matte Black body wrap, with Carbon Sigma blades & Black leatherupholstery.. Astonishing factory specification, with over £10k in optional extras!! Sat Nav.. Front & rear parking assist.. Alcantara headlining.. Mag ride suspension.. Bang & Olufsen premium sound system.. 19" Twin spoke alloy wheels, in Gloss Black.
 Please visit our website for full specification and hi-resolution photographs.... Junction 17 Cars is a unique family run business, with all our cars HPI clear, and individually sourced, with service history, and fantastic features such as.... Outstanding Audi main dealer service history (with 6 x service stamps in the book).... Unbeatable flexible finance deals available with same-day approval.... UK's best part 

Meta detial: # 25  id: 201911074114226   desc: Price: £44,999. White, Here At Car Capital We Have A Beautiful 2011 Audi R8, this gem Has Only Covered 45,000 Miles And Has Full Service History. This Car Is In Immaculate Condition Throughout And Drives Superb! It Has Been Maintained Very Well, Last Serviced Carried Out In September  and  4 New Bridgestone Tyres Recently Fitted. The Bodywork Is In Immaculate Condition, 2 Keys  and  Complete Owners Pack Present  ~~~~~~~~~~~~~~~~~~~~~~~~~~  This R8 Is Packed With Extras Making It Even More Desirable! 19 Black Alloy Wheels, Carbon Side Blades, Colour Sat Nav, Reverse Camera, Parking Aid, MMI Port, Bluetooth, Paddle Shift, Sport Mode, B and O Premium Sound System, Full Black Leather Interior, Heated Seats, Auto Lights, Rain Sensoring Wipers, Trip Computer  ~~~~~~~~~~~~~~~~~~~~~~~~~~  -HPI All Of Our Vehicles Are HPI Clear  -FINANCE  and  PCP  We Work Closely With Lenders So We Are Able To Offer Low Rate Flexible Finance  and  Pcp Packages To 

Meta detial: # 33  id: 201609057450945   desc: Price: £49,000. HPi Clear Low Deposit Finance Arranged PX Welcome
inspection of Advert 34 of 90 for more details
url: https://www.autotrader.co.uk/classified/advert/201910103167550
Meta detial: # 34  id: 201910103167550   desc: Price: £49,950. 4.2 litre V8 with the updated 7 Speed S-tronic gearbox. White paint with Silver side blade and Black wheels. One owner since new with very low mileage. Black styling pack and illuminated engine bay. Full LED headlights and LED rear indicators. New Pirelli tyres have covered less than 1000 miles.  Excellent condition and drives immaculately., Next MOT due 19/12/2019, Electric windows, Air conditioning, Satellite navigation, Parking aid, CD player, Bluetooth, Leather trim, Heated seats, Height adjustable driver's seat, Power steering, Traction control, Central locking, Alarm, Immobiliser, Driver's airbags, Side airbags, Passenger airbags. White, 1 owner, £49,950
inspection of Advert 35 of 90 for more d

...

Meta detial: # 53  id: 201909122150834   desc: Price: £62,950. Black, 19" '5-arm double-spoke' design alloy wheels (19" x 8.5J 235/35 front tyres and 19" x 11J 295/30 R19 rear tyres), Front tires 235/35 R19 91Y rear tires 295/30 R19 100Y, 19" '5-arm double-spoke' design alloy wheels (19" x 8.5J 235/35 front tyres and 19" x 11J 295/30 R19 rear tyres), Mobile telephone preparation - Low with Bluetooth interface, Voice dialogue system, Garage door opener (HomeLink&reg;), Sports suspension with dynamically tuned spring and damper combination, CD-changer, Cruise control, R8 3-spoke contoured flat-bottomed multifunction Sports steering wheel, Electrically adjustable seats, Heated seats, Satellite Navigation system - DVD-based, Headlight washers, Electronic climate control, Driver's Information System (DIS), Established since 1936, £62,950
inspection of Advert 54 of 90 for more details
url: https://www.autotrader.co.uk/classified/advert/201910103171596
Meta detial: # 54  id: 201910103171596  

Meta detial: # 63  id: 201910233626824   desc: Price: £76,990. We're proud to offer this 2016/16 Audi R8 V10+ finished in Suzuka Grey with Black leather. This vehicle has every factory carbon fibre option ticked and has a full Audi Service History.

Options,

Suzuka Grey
Black leather 
Gloss Black 20-inch wheels
Silver carbon ceramic brakes
Carbon fibre side blades
Carbon fibre side vents
Carbon fibre rear spoiler 
Carbon fibre rear diffuser 
Carbon fibre door mirrors
Carbon fibre front splitter 
Bang & Olufsen sound system
14-way electric sports bucket seats
Carbon fibre air-conditioning vents
Carbon fibre driving zone 
Carbon fibre centre console
Audi Multimedia 
Rear view camera with reversing sensors
Perforated leather steering wheel
Sports exhaust system
Heated seats 
Cruise control
inspection of Advert 64 of 90 for more details
url: https://www.autotrader.co.uk/classified/advert/201910032899848
Meta detial: # 64  id: 201910032899848   desc: Price: £76,990. PRESTIGE CARS ARE DELIG

Meta detial: # 75  id: 201902285384938   desc: Price: £84,900. Audi R8 V10+ Coupe in metallic Mythos black protected in warranted XPEL Satin PPF! Black leather sport seats with sound and comfort pack, carbon fibre interior trim package and a Bang & Olufsen upgraded sound system. This brilliant R8 V10+ comes with Carbon fibre side blades, carbon fibre rear wing, sport plus pack, sports exhaust, larger fuel tank and 20 inch black multi spoke alloys.

Walk around video is available upon request. More pictures and information can be found on our website www.gvelondon.com 
12 month warranty available through JBR/Aldermore/Black Horse. All of our vehicles undergo a full visual and mechanical health check by GVE Servicing and preparation by GVE detailing. GVE Detailing offers a personal car customisation option through ceramic coating, paint protection film, vehicle wraps and resprays.

inspection of Advert 76 of 90 for more details
url: https://www.autotrader.co.uk/classified/advert/20191021

Meta detial: # 81  id: 201910283768915   desc: Price: £89,990.  Optional extras
       1. Equipment and design packages
          -[S9T] Special vehicle for launch
          -[4ZH] Titanium Black Styling Pack
       2. Paint finishes
          -[6H6] Sideblade - Mythos Black
       3. Wheels/tyres
          -[45F] 20" 10-spk Y gloss black alloy
       23. Suspensions/brakes
           [PC2] Red brake calipers
             -[1KY] Disk brakes in rear
             -[1LJ] Disk brakes in front
          -[VF2] Stainless steel pedals
       10. Seats/seating comfort
           [PS5] R8 Bucket seats
             -[3PF] Power height adjustment for both front seats
             -[Q4P] Bucket seats
       11. Seat upholstery/leather trim
          -[N0Q] Fine Nappa Leather Perforated
          -[7HB] Leather package
       19. Radio and TV systems
          -[9VS] Bang and Olufsen Sound system
       22. Assistance systems
          -[KA2] Reversing camera
       9. Other exterior equipment
    

Meta detial: # 88  id: 201911044003608   desc: Price: £97,450. Blue, Boss Motor Company are delighted to offer this stunning Audi R8 V10+ Coupe.Presented in Ara Blue Metallic with full Black leather interior and contrast stitch to match body colour.Complimented by Full Audi Main Dealer Service History.The specification of this car includes: Carbon sideblade, Sports package plus, Sports exhaust system, Dynamic steering, Variable magnetic ride, Diamond stitched seats, Comfort and sound package (B&O), Driver assistance pack, Cruise control, Reversing camera, Garage door opener, Heated seats, High beam assist, Navigation system, Power adjustable sports seats with pneumatic bolsters, Keyless entry, and more!- VIEWING STRICTLY BY APPOINTMENT ONLY - PART EXCHANGE WELCOME- FINANCE AVAILABLE THROUGH OUR WEBSITE - RAC WARRANTYREGISTRATION NUMBER AVAILABLE UPON REQUEST. PLEASE CALL FOR CONFIRMATION.Disclaimer: Whilst every effort has been taken to ensure accuracy of all our vehicles information, 

In [9]:
print(spec_list[85].title)
print(spec_list[85].id)
print(spec_list[85].price)
print(spec_list[85].miles)
print(spec_list[85].url)
print(spec_list[85].descriptionRaw)



Audi R8 Coup- V10 plus 5.2 FSI quattro 610 PS S tronic 2dr

201910263730111
96950
8573
https://www.autotrader.co.uk/classified/advert/201910263730111
Price: £96,950. Four wheel-drive, Rear-view camera, Sport exhaust system, 20" '10-spoke Y' design forged alloy wheels in gloss Anthracite black with 8.5J 245/30 R20 front tyres and 11.0J 305/30 R20 rear tyres, Audi Connect Infotainment Services, MMI Navigation Plus with MMI Touch, Fine Nappa leather perforated upholstery and trim, Interior Lighting Pack, DAB radio, Electromechanical power steering, Bucket seats, Audi Music Interface, Ceramic brakes, Anti-theft wheel bolts, R8 3-spoke, flat-bottomed, multi-function, perforated performance leather steering wheel with four operating satellites, Electric height adjustment for bothfront seats, Heated seats, Advanced key, Side airbag front with curtain airbag, Exclusive carbon package, Exterior mirrors, electrically adjustable, heated, folding and auto-dimming with integrated LED indicator, An

In [10]:
json_data = {"source": "autotrader", "timestamp":"20191110"}
json_data['ads'] = []

for advert in spec_list:
    json_data['ads'].append({
        'id': advert.id,
        'title': advert.title.strip(),
        'price': advert.price,
        'reg'  : advert.reg_year_month,
        'body_type' : advert.body_type,
        'miles' : advert.miles,
        'cc'    :  advert.cc,
        'tran'  : advert.transmission,
        'fuel'  : advert.fuel,
        'url'   : advert.url,
        'description_raw' : advert.descriptionRaw,
        'other1' : advert.other1,
        'other2' : advert.other2,
        'other3' : advert.other3

        
    })

import json
with open('car_science_autodtrader_20191110.json', 'w') as outfile:
    json.dump(json_data, outfile)
        
       
    